In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
# NOTE: the next import is only valid for scikit-learn version <= 0.17
# for scikit-learn >= 0.18 use:
from sklearn.model_selection import train_test_split
import pickle
%matplotlib inline
# from sklearn.cross_validation import train_test_split

In [2]:
# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

# Define a function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, cspace='RGB', spatial_size=(32, 32),
                        hist_bins=32, hist_range=(0, 256), orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)[:,:,0:3]
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif color_space == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image)      
        # Apply bin_spatial() to get spatial color features
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        # Apply color_hist() also with a color space option now
        hist_features = color_hist(feature_image, nbins=hist_bins, bins_range=hist_range)
        # Call get_hog_features() with vis=False, feature_vec=True
        if hog_feat == True:
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)

            # Append the new feature vector to the features list
            features.append(np.concatenate((spatial_features, hist_features,hog_features)))
        else:
               features.append(np.concatenate((spatial_features, hist_features)))
    print (np.shape(features))
    # Return list of feature vectors
    return features


In [3]:
cars =[]
cars.extend(glob.glob('./train_images/vehicles/vehicles/GTI_Far/*.png'))
cars.extend(glob.glob('./train_images/vehicles/vehicles/GTI_Left/*.png'))
cars.extend(glob.glob('./train_images/vehicles/vehicles/GTI_MiddleClose/*.png'))
cars.extend(glob.glob('./train_images/vehicles/vehicles/GTI_Right/*.png'))
cars.extend(glob.glob('./train_images/vehicles/vehicles/KITTI_extracted/*.png'))

In [4]:
notcars=[]
notcars.extend(glob.glob('./train_images/non-vehicles/non-vehicles/GTI/*.png'))
notcars.extend(glob.glob('./train_images/non-vehicles/non-vehicles/Extras/*.png'))
notcars.extend(glob.glob('./train_images/non-vehicles/non-vehicles/RE/*.png'))

In [5]:
# test_glob = (glob.glob('./train_images/non-vehicles/non-vehicles/RE/*.png'))
# print (len(test_glob))
# file = test_glob[5]
# image = mpimg.imread(file)[:,:,0:3]
# print (np.shape(image))
# plt.imshow(image[:,:,1])

In [8]:
# TODO play with these values to see how your classifier
# performs under different binning scenarios
spatial = 32
histbin = 32
colorspace = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9
pix_per_cell = 16
cell_per_block = 2
hog_channel = "ALL"# Can be 0, 1, 2, or "ALL"

#Pickle these settings with out classifier
color_space = colorspace # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
# orient = 9  # HOG orientations
# pix_per_cell = 8 # HOG pixels per cell
# cell_per_block = 2 # HOG cells per block
# hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (spatial, spatial) # Spatial binning dimensions
hist_bins = histbin   # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off

settings = (color_space,orient,pix_per_cell,cell_per_block,
            hog_channel,spatial_size,hist_bins,spatial_feat,hist_feat,hog_feat)

#predicted size
# spatial feat: 16*16 = 256
# hist feat: 24*24 = 576
# HOG feat: 2 * 2 * 7 * 7 *9 *3 = 5292
# Total = 6124

#Measured: 6132 ?? close..

t=time.time()
car_features = extract_features(cars, cspace=colorspace, spatial_size=(spatial, spatial),
                        hist_bins=histbin, hist_range=(0, 256), orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
notcar_features = extract_features(notcars, cspace=colorspace, spatial_size=(spatial, spatial),
                        hist_bins=histbin, hist_range=(0, 256), orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
# test_features = extract_features(test_glob, cspace=colorspace, spatial_size=(spatial, spatial),
#                         hist_bins=histbin, hist_range=(0, 256), orient=orient, 
#                         pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
#                         hog_channel=hog_channel)

t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract features...')

# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

(8792, 4140)
(9579, 4140)
44.67 Seconds to extract features...


In [9]:
print (len(notcars))

9579


In [10]:
# print(np.size(scaled_X))
# print(np.size(y))
print (np.shape(car_features))
print (np.shape(notcar_features))
# print (np.shape(test_features))

(8792, 4140)
(9579, 4140)


NameError: name 'test_features' is not defined

In [11]:
#Quick test extracting features for one image
feat = extract_features([cars[0]], cspace='RGB', spatial_size=(spatial, spatial),
                        hist_bins=histbin, hist_range=(0, 256), orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)

(1, 4140)


In [12]:
#Check the size of one image
print (np.size(feat))

4140


In [13]:
# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

In [14]:
print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')


Using: 9 orientations 16 pixels per cell and 2 cells per block
Feature vector length: 4140
7.87 Seconds to train SVC...


In [15]:
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = 15
print('My SVC predicts:     ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

Test Accuracy of SVC =  0.9845
My SVC predicts:      [ 0.  1.  0.  1.  1.  1.  0.  1.  0.  0.  1.  0.  0.  0.  1.]
For these 15 labels:  [ 0.  1.  0.  0.  1.  1.  0.  1.  0.  0.  1.  0.  0.  0.  1.]
0.00283 Seconds to predict 15 labels with SVC


In [16]:
pickle_out = ((svc,X_scaler,settings))
pickle.dump(pickle_out, open( "trainedYCrCb32-32-32-A16svc.p", "wb" ) )